In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import time
#from sklearn.inspection import plot_partial_dependence
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_gaussian_quantiles
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from collections import Counter
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.inspection import permutation_importance
import pickle
import shap
from sklearn.metrics import roc_curve, auc
import matplotlib.cm as cm
from sklearn.model_selection import cross_validate
from sklearn.cluster import KMeans

def calc_metrics(y_test, preds):
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1score = f1_score(y_test, preds)
    return (accuracy, precision, recall, f1score)

def model_predict(model, x_train, y_train, x_test, y_test):
    model = model.fit(x_train, y_train)
    y_predict_train = model.predict(x_train)
    y_predict_test = model.predict(x_test)
    return accuracy_score(y_predict_train, y_train), accuracy_score(y_predict_test, y_test), y_predict_test

def sort_feature(feature_names, feature_vals):
    assert len(feature_names) == len(feature_vals)
    x_and_y = [(x, y) for x, y in zip(feature_names, feature_vals)]
    x_and_y.sort(key = lambda x: -x[1])
    return [item[0] for item in x_and_y], [item[1] for item in x_and_y]

def get_feature_importance(model, x_test, y_test):
    perm_importance = permutation_importance(model, x_test, y_test)
    perm_sorted_idx = perm_importance.importances_mean.argsort()
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(x_test)
    return (model.feature_importances_, perm_sorted_idx, shap_values)

def machine_learning(x, y, split_ratio=0.3, random_seed=28):
    xstd = x.std()
    xmean = x.mean()

    x = (x - x.mean()) / x.std()
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=split_ratio, random_state=random_seed,
                                                        shuffle=True)
    xtestvalue = x_test * xstd + xmean

    map = {}

    # LightGBM
    print("Starting LightGBM。。。")
    time1 = time.time()
    gbm = lgb.LGBMClassifier(random_state=random_seed)
    score_train, score_test, y_predict_test = model_predict(gbm, x_train, y_train, x_test, y_test)
    y_pred_proba = gbm.predict_proba(x_test)[:, 1]

    dt_featureimprotance, permsortedidx, shapevalues = get_feature_importance(gbm, x_test, y_test)

    map['LightGBM'] = (
    dt_featureimprotance, permsortedidx, xstd, xmean, xtestvalue, shapevalues, x_test, y_test, y_pred_proba)

    calc_metrics(y_test, y_predict_test)

    return map

def K_Means(df, cluster_raw, k):
    # 提取特征列
    X = df[cluster_raw].values

    # 初始化KMeans对象并进行拟合
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(X)

    # 获取聚类中心和标签
    centroids = kmeans.cluster_centers_
    labels = kmeans.labels_

    # 将聚类标签添加为DataFrame的一个新列
    df['cluster_label'] = labels

    # 返回带有聚类标签的DataFrame、聚类中心和标签
    return df, centroids, labels

result_map = {}

csv_file_path = r"C:\Users\29688\Desktop\study-2\sj_T1\T1.xlsx"
df = pd.read_excel(csv_file_path)
# 设置特征列，并且根据这些特征进行聚类
cluster_raw = ['LS', 'ladder', 'PMWB']
k = 3

# 调用K_Means函数
df, centroids, labels = K_Means(df, cluster_raw, k)

# 打印每个编号对应的聚类标签
print("查看每个编号对应的聚类标签：")
print(df[['ID', 'cluster_label']])  # 假设'Number'列存在于df中

# 打印聚类标签和聚类中心
print("聚类标签:", labels)
print("聚类中心:", centroids)

# 根据聚类标签创建二进制变量列
df['high_binary'] = (df['cluster_label'] == 2).astype(int)  # 2表示高类别
df['middle_binary'] = (df['cluster_label'] == 0).astype(int)  # 1表示中类别
df['low_binary'] = (df['cluster_label'] == 1).astype(int)  # 0表示低类别

# 保存修改后的 DataFrame 到 CSV 文件
df.to_excel(csv_file_path)

# 打印修改后的 DataFrame 头部，以确认结果
print(df.head())

df = pd.read_excel (r"C:\Users\29688\Desktop\预处理_T1匹配_1.xlsx")
columns_to_replace = ['PRO', 'gender', 'age', 'height', 'weight','marriage', 'kid', 'Kage',
             'BKage', 'LKage', 'education','shifan', 'xueduan', 'BZR',
             'subject', 'title', 'Tage', 'DEP', 'ANX', 'LON', 'RES', 'STR', 'sleep',
             'rise', 'commute', 'work', 'mediation', 'sport', 'location_type', 'school_type']

# 遍历需要替换的列
for column in columns_to_replace:
    # 尝试将列中的字符串转换为数值，并将空字符串替换为NaN
    df[column] = pd.to_numeric(df[column].replace('', np.nan), errors='coerce')

    # 计算该列的中位数
    median = df[column].median()

    # 使用中位数替换NaN值
    # 这里不需要使用inplace=True，因为我们已经直接赋值给df[column]
    df[column] = df[column].fillna(median)

# 将修改后的DataFrame保存回CSV文件
df.to_excel(r"C:\Users\29688\Desktop\预处理_T1匹配_1.xlsx")

df = pd.read_excel (r"C:\Users\29688\Desktop\预处理_T1匹配_1.xlsx")
# 自变量
df_src = df[['PRO', 'gender', 'age', 'height', 'weight','marriage', 'kid', 'Kage',
             'BKage', 'LKage', 'education','shifan', 'xueduan', 'BZR',
             'subject', 'title', 'Tage', 'DEP', 'ANX', 'LON', 'RES', 'STR', 'sleep',
             'rise', 'commute', 'work', 'mediation', 'sport', 'location_type', 'school_type']].astype(float)
# 因变量
df_tgt = df[['high_binary']]
print("Data prepare done")

for i in range(1):
    result_map[i] = machine_learning(df_src, df_tgt.values.ravel(), random_seed=i)

    for model_name, vals in result_map[0].items():
        shap.summary_plot(vals[-4], vals[-5])
        save_path_1 = 'high_shap_value_plot.png'
    # 保存图片
        plt.savefig(save_path_1, bbox_inches='tight')
    import seaborn as sns

    sns.set_theme(style="darkgrid")


def cross_validation(model, _X, _y, _cv=10):
    '''Function to perform 5 Folds Cross-Validation
     Parameters
     ----------
    model: Python Class, default=None
            This is the machine learning algorithm to be used for training.
    _X: array
         This is the matrix of features.
    _y: array
         This is the target variable.
    _cv: int, default=10
        Determines the number of folds for cross-validation.
     Returns
     -------
     The function returns a dictionary containing the metrics 'accuracy', 'precision',
     'recall', 'f1' for both training set and validation set.
    '''
    _scoring = ['accuracy', 'precision', 'recall', 'f1']
    results = cross_validate(estimator=model,
                             X=_X,
                             y=_y,
                             cv=_cv,
                             scoring=_scoring,
                             return_train_score=True)
    return {"Training Accuracy scores": results['train_accuracy'],
            "Mean Training Accuracy": results['train_accuracy'].mean() * 100,
            "Validation Accuracy scores": results['test_accuracy'],
            "Mean Validation Accuracy": results['test_accuracy'].mean() * 100
            }

def plot_result(x_label, y_label, plot_title, train_data, val_data):
    '''Function to plot a grouped bar chart showing the training and validation
      results of the ML model in each fold after applying K-fold cross-validation.
     Parameters
     ----------
     x_label: str,
        Name of the algorithm used for training e.g 'Decision Tree'

     y_label: str,
        Name of metric being visualized e.g 'Accuracy'
     plot_title: str,
        This is the title of the plot e.g 'Accuracy Plot'

     train_result: list, array
        This is the list containing either training precision, accuracy, or f1 score.

     val_result: list, array
        This is the list containing either validation precision, accuracy, or f1 score.
     Returns
     -------
     The function returns a Grouped Barchart showing the training and validation result
     in each fold.
    '''

    # Set size of plot
    plt.figure(figsize=(12, 6))
    labels = ["1st Fold", "2nd Fold", "3rd Fold", "4th Fold", "5th Fold", "6st Fold", "7nd Fold", "8rd Fold",
              "9th Fold", "10th Fold"]
    X_axis = np.arange(len(labels))
    ax = plt.gca()
    plt.ylim(0.40000, 1)
    plt.bar(X_axis - 0.2, train_data, 0.4, color='blue', label='Training')
    plt.bar(X_axis + 0.2, val_data, 0.4, color='red', label='Validation')
    plt.title(plot_title, fontsize=24)
    plt.xticks(X_axis, labels)
    plt.xlabel(x_label, fontsize=18)
    plt.ylabel(y_label, fontsize=18)
    plt.legend()
    plt.grid(True)
    save_path_2 = 'high_grouped_barplot_accuracy.png'
    plt.savefig(save_path_2)
    plt.show()

x_train, x_test, y_train, y_test = train_test_split(df_src, df_tgt.values.ravel(), test_size = 0.3, random_state = 28, shuffle = True)

gbm = lgb.LGBMClassifier(random_state=28)
gbm_result = cross_validation(gbm, x_test, y_test, 10)
print(gbm_result)

model_name = "lightgbm"
plot_result(model_name,
            "Accuracy",
            "High Well Being string Victimization: Accuracy scores in 10 Folds",
            gbm_result["Training Accuracy scores"],
            gbm_result["Validation Accuracy scores"])